In [42]:
import torch

from pinder.core import get_index, PinderSystem
from pinder.core import PinderLoader
from pinder.core.loader import filters
from pinder.core.index.system import PinderSystem
from pinder.core import get_pinder_location
import re
import pandas as pd
import csv
import os

UNIPROT_IDS_PATH="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/uniprot_ids_1024_512.lst"
UNIPROT_LINEAGES_PATH="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/uniprot_lineages_1024_512.tsv"
POSITIVES_DIR="/scicore/home/schwede/pudziu0000/projects/gLM/data/PINDER/eubacteria_5_1024_512/"

def get_system(system_id: str) -> PinderSystem:
    return PinderSystem(system_id)

def select_ids(max_length=1024, max_length_per_monomer=400):
    base_filters = [
        filters.FilterByMissingHolo(),
        filters.FilterSubByContacts(min_contacts=5, radius=10.0, calpha_only=True),
        filters.FilterDetachedHolo(radius=12, max_components=2),
    ]
    sub_filters = [
        filters.FilterSubByAtomTypes(min_atom_types=4),
        filters.FilterByHoloOverlap(min_overlap=5),
        filters.FilterByHoloSeqIdentity(min_sequence_identity=0.8),
        filters.FilterSubRmsds(rmsd_cutoff=7.5),
        filters.FilterDetachedSub(radius=12, max_components=2),
    ]
    split_pinder_ids = {}
    split_uniprot_ids = {}
    train_index = None
    for split in ["train", "val", "test"]:
        loader = PinderLoader(
            split=split,
            monomer_priority="holo",
            base_filters=base_filters,
            sub_filters=sub_filters,
        )
        index = loader.index.merge(loader.metadata, on="id", how="left")
        index["length"] = index["length1"] + index["length2"]
        index["resolution"] = index["resolution"].astype("float32")
        index = index[
            (index["length"] + 2 <= max_length)
            & (index["length1"] <= max_length_per_monomer)
            & (index["length2"] <= max_length_per_monomer)
            & (index["label"] == "BIO")
        ]
        if(split == "train"): train_index = index
        pinder_ids = list(index["id"])
        uniprot_ids = set(list(index["uniprot_R"]) + list(index["uniprot_L"]))
        split_pinder_ids[split] = pinder_ids
        split_uniprot_ids[split] = uniprot_ids
    return split_pinder_ids, split_uniprot_ids, train_index

def reduce_train_redundancy(ids, index, max_per_cluster=5):
    # ids - list of ids of the training set
    # index - index of the training set
    ids = list(
        set(
            index.sort_values(
                ["resolution", "id"],
                ascending=[True, True],
            )
            .groupby("cluster_id", observed=False)
            .head(max_per_cluster)["id"]
        )
    )
    return ids
    

In [28]:
os.environ["PINDER_BASE_DIR"] = "/scicore/home/schwede/durair0000/.local/share/"
get_pinder_location()

PosixPath('/scicore/home/schwede/durair0000/.local/share/pinder/2024-02')

In [29]:
pinder_ids, uniprot_ids, train_index = select_ids(max_length_per_monomer=512)

In [30]:
# Checking number of elements in each split
print(len(pinder_ids['train']),
      len(pinder_ids['val']),
      len(pinder_ids['test'])
)

print(len(uniprot_ids['train']),
      len(uniprot_ids['val']),
      len(uniprot_ids['test'])
)

675729 1799 1767
20907 1937 1906


In [31]:
# Selecting that that both defined UniProt IDs
pinder_ids['train'] = [row for row in pinder_ids['train'] if not re.search(r'UNDEFINED', row, re.IGNORECASE)]
pinder_ids['val'] = [row for row in pinder_ids['val'] if not re.search(r'UNDEFINED', row, re.IGNORECASE)]
pinder_ids['test'] = [row for row in pinder_ids['test'] if not re.search(r'UNDEFINED', row, re.IGNORECASE)]

uniprot_ids['train'].remove("UNDEFINED")
uniprot_ids['val'].remove("UNDEFINED")
uniprot_ids['test'].remove("UNDEFINED")

In [32]:
# Checking number of elements in each split after removal of "UNDEFINED"
print(len(pinder_ids['train']),
      len(pinder_ids['val']),
      len(pinder_ids['test'])
)

print(len(uniprot_ids['train']),
      len(uniprot_ids['val']),
      len(uniprot_ids['test'])
)

617197 1748 1688
20906 1936 1905


In [33]:
# Save UniProt IDs to a TXT file for "ID mapping"
with open(UNIPROT_IDS_PATH, 'w') as f:
    for split in ["train", "val", "test"]:
        for line in uniprot_ids[split]:
            f.write(f"{line}\n")

In [ ]:
# Manually run the ID mapping on the UniProt

In [47]:
# Determine, which UniProt IDs are belonging to eubacteria with labelled phylum
lineages = pd.read_csv(UNIPROT_LINEAGES_PATH, sep='\t')

eubacteria_phyla = {}

for index, row in lineages.iterrows():
    if isinstance(row['Taxonomic lineage (Ids)'], str) and "2 (superkingdom)" in row['Taxonomic lineage (Ids)']: 
        match = re.search(r'(\d+) \(phylum\)', row['Taxonomic lineage (Ids)'])
        if match:
            phylum_tax_id = match.group(1)
            eubacteria_phyla[row['From']] = phylum_tax_id

In [50]:
# Determine, which PINDER IDs contain both bacterial proteins
eubacteria_pinder_ids = {'train': [], 'val': [], 'test': []}
eubacteria_pinder_phyla = {}

for split in ["train", "val", "test"]:
    for id in pinder_ids[split]:
        uniprot1 = id.split("--")[0].split("_")[-1]
        uniprot2 = id.split("--")[1].split("_")[-1]
        if(uniprot1 in eubacteria_phyla.keys() and uniprot2 in eubacteria_phyla.keys()):
            eubacteria_pinder_ids[split].append(id)
            eubacteria_pinder_phyla[id] = [eubacteria_phyla[uniprot1], eubacteria_phyla[uniprot2]]

In [51]:
print(len(eubacteria_pinder_ids['train']),
      len(eubacteria_pinder_ids['val']),
      len(eubacteria_pinder_ids['test'])
)

86221 906 885


In [43]:
# Remove redundancy from the training set
mask = train_index['id'].isin(eubacteria_pinder_ids['train'])
eubacteria_train_index = train_index[mask]
eubacteria_pinder_ids['train'] = reduce_train_redundancy(eubacteria_pinder_ids['train'], eubacteria_train_index, max_per_cluster=5)

In [46]:
# Number of elements in each split after redundancy removal
print(len(eubacteria_pinder_ids['train']),
      len(eubacteria_pinder_ids['val']),
      len(eubacteria_pinder_ids['test'])
)

8283 906 885


In [53]:
# Save IDs of the positives and phyla
for split in ["train", "val", "test"]:
    split_data = []
    for id in eubacteria_pinder_ids[split]:
        split_data.append({"pinder_id": id, "phylum_1": eubacteria_pinder_phyla[id][0], "phylum_2": eubacteria_pinder_phyla[id][1]})
    with open(f"{POSITIVES_DIR}/{split}.csv", 'w', newline='') as csvfile:
        fieldnames = ['pinder_id', 'phylum_1', 'phylum_2']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(split_data)

In [ ]:
# Make negative pairs